In [375]:
import os
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt

In [376]:
IMAGES_PATH = Path('../docs/imgs/')
DATA_PATH = Path('../data/project/')

IMAGES_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [377]:
SEED = 42
BATCH_SIZE = 64

In [378]:
def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

seed_all(SEED)

# Загрузка данных

In [379]:
from uu import encode
import pandas as pd

def load_data(file_path: Path) -> pd.DataFrame:
    data = pd.read_excel(file_path)    
    return data

In [380]:
dataframe = load_data(os.path.join(DATA_PATH, "data_1.xlsx"))
print("Number of rows and columns in the train data set:", dataframe.shape)

Number of rows and columns in the train data set: (11475, 8)


d:\Projects\NLP_course\.venv\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [381]:
dataframe.head(2)

,responsible_person,type_problem,topic,categoria,region,source,target,context
0,Администрация Химки,Устранение проблемы,"Неудовлетворительное качество товара, оказания...","Торговля, товары и услуги",Орехово-Зуевский,"Цифровизация услуг в любых сферах, а особенно ...",Здравствуйте! На трубопроводе центрального ото...,"[id181153628|Ольга], совершенно верно! Каждый..."
1,Администрация Химки,Устранение проблемы,"Неудовлетворительное качество товара, оказания...","Торговля, товары и услуги",Орехово-Зуевский,"Цифровизация услуг в любых сферах, а особенно ...","Здравствуйте! Специалисты ТСК ""Мосэнерго"" пров...","[id181153628|Ольга], совершенно верно! Каждый..."


In [382]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11475 entries, 0 to 11474
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   responsible_person  11475 non-null  object
 1   type_problem        11475 non-null  object
 2   topic               11475 non-null  object
 3   categoria           11475 non-null  object
 4   region              11475 non-null  object
 5   source              11150 non-null  object
 6   target              11465 non-null  object
 7   context             11475 non-null  object
dtypes: object(8)
memory usage: 717.3+ KB


In [383]:
dataframe.describe()

,responsible_person,type_problem,topic,categoria,region,source,target,context
count,11475,11475,11475,11475,11475,11150,11465,11475
unique,181,11,120,11,56,5534,9821,5694
top,Александр Αлексеев,Устранение проблемы,-,ЖКХ,Другие регионы,Это ситуация в доме 15/2 - результат полного о...,Здравствуйте! Спасибо за Ваш вопрос. В микрора...,"[id4847589|Александр], кто ответит за нанесён..."
freq,717,8365,1663,4848,2513,24,111,24


## Препроцессинг

## Удаление пропусков

In [384]:
dataframe = dataframe.dropna(subset=['target', 'source'])
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11140 entries, 0 to 11474
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   responsible_person  11140 non-null  object
 1   type_problem        11140 non-null  object
 2   topic               11140 non-null  object
 3   categoria           11140 non-null  object
 4   region              11140 non-null  object
 5   source              11140 non-null  object
 6   target              11140 non-null  object
 7   context             11140 non-null  object
dtypes: object(8)
memory usage: 783.3+ KB


### Кодирование меток

In [385]:

from sklearn.preprocessing import LabelEncoder

work_dataframe = dataframe.copy()

class CategoricalLabelEncoder():
    def __init__(self, dataframe: pd.DataFrame, categorical_columns: list[str]) -> None:
        self.encoders = {}
        
        for column in categorical_columns:
            encoder = LabelEncoder()
            dataframe[column] = encoder.fit_transform(dataframe[column])
            self.encoders[column] = encoder
        
    def decode(self, label: str, code: list[int]):
        return self.encoders[label].inverse_transform([code])
    
    def get_classes(self, label):
        categories_list = self.encoders[label].classes_
        print("Список категорий в 'categoria':", categories_list)
        return categories_list

categorical_columns = ['responsible_person', 'type_problem', 'topic', 'categoria', 'region']
datafarme_encoders = CategoricalLabelEncoder(work_dataframe, categorical_columns)

datafarme_encoders.get_classes('type_problem')

print(datafarme_encoders.decode('type_problem', 10))

Список категорий в 'categoria': ['Благодарности' 'Комментарии' 'Критика власти' 'Неопределенное сообщение'
 'Ответы' 'Повторная жалоба' 'Предложение/инициатива/идея'
 'Просьба проинформировать' 'Семьи военнослужащих' 'Соболезнования'
 'Устранение проблемы']
['Устранение проблемы']


In [386]:
work_dataframe.head(4)

,responsible_person,type_problem,topic,categoria,region,source,target,context
0,30,10,79,9,34,"Цифровизация услуг в любых сферах, а особенно ...",Здравствуйте! На трубопроводе центрального ото...,"[id181153628|Ольга], совершенно верно! Каждый..."
1,30,10,79,9,34,"Цифровизация услуг в любых сферах, а особенно ...","Здравствуйте! Специалисты ТСК ""Мосэнерго"" пров...","[id181153628|Ольга], совершенно верно! Каждый..."
2,30,10,79,9,34,"Цифровизация услуг в любых сферах, а особенно ...","Здравствуйте! Конкретизируйте, пожалуйста, Ваш...","[id181153628|Ольга], совершенно верно! Каждый..."
3,30,10,79,9,34,"Цифровизация услуг в любых сферах, а особенно ...",Здравствуйте! Спасибо за Ваш вопрос. В микрора...,"[id181153628|Ольга], совершенно верно! Каждый..."


# Датасет

In [387]:
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer

class CustomDataset(Dataset):
    def __init__(self, 
                 dataframe: pd.DataFrame,
                 tokenizer: GPT2Tokenizer,
                 max_length=512):
        
        self.dataframe = dataframe
        
        self.type_ids = torch.tensor(
            dataframe['type_problem'].astype(int).tolist()
        )

        self.source_encodings = tokenizer(
            dataframe['source'].tolist(), 
            padding='max_length', 
            truncation=True, 
            max_length=max_length, 
            return_tensors="pt"
        )
        self.target_encodings = tokenizer(
            dataframe['target'].tolist(), 
            padding='max_length', 
            truncation=True, 
            max_length=max_length, 
            return_tensors="pt"
        )      

    def __len__(self):
        return len(self.type_ids)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.source_encodings.items()}
        item['target_ids'] = self.target_encodings['input_ids'][idx]
        item['target_attention_mask'] = self.target_encodings['attention_mask'][idx]
        item['type_ids'] = self.type_ids[idx]
        return item


# Модель
В модели `CustomGPT2Model` логиты будут иметь размеры зависящие от следующих факторов:

1. **Количество токенов в входной последовательности**: Обозначим количество токенов как `T`. Это количество токенов, которое вы подаете в модель в `input_ids`. Также это число будет равно длине последовательности, которая используется для генерации выхода.

2. **Размер словаря модели GPT-2**: Обозначим размер словаря как `V`. Это количество уникальных токенов, которые модель может генерировать. Этот параметр зависит от конфигурации предобученной модели GPT-2, которую вы используете.

### Размеры тензоров в вашей модели:

- **`input_ids`**: Тензор размером `[B, T]`, где `B` — размер батча, `T` — количество токенов в каждой последовательности входных данных.

- **`type_ids`**: Тензор размером `[B]`, содержащий идентификаторы типов сообщений для каждого элемента в батче. Этот тензор расширяется и повторяется, чтобы соответствовать длине `T`.

- **`attention_mask`**: Тензор размером `[B, T]`, который указывает на то, какие токены являются значимыми для каждой последовательности.

- **`combined_embeds` и `inputs_embeds`**: Тензоры размером `[B, T, H]`, где `H` — размер скрытого состояния в модели GPT-2.

- **`logits`** после применения `lm_head`: Тензор размером `[B, T, V]`, где каждый элемент в тензоре представляет логиты для каждого токена в каждой последовательности батча для каждого возможного токена в словаре.

### Как это работает в вашей `forward` функции:

1. **Токен-встраивания** (`wte`) и **встраивания типа** комбинируются и проходят через линейный слой (`combined_linear`).

2. **Встраивания** передаются в основную модель GPT-2, которая возвращает последние скрытые состояния.

3. **Последние скрытые состояния** преобразуются в логиты с помощью `lm_head`, которые представляют собой вероятности следующего токена для каждого токена в последовательности.

Таким образом, если вы передаете `input_ids` размером `[5, 5]` (5 последовательностей по 5 токенов каждая), вы получите `logits` размером `[5, 5, V]`, где `V` — размер словаря вашей модели GPT-2. Эти логиты используются для выбора следующего токена в процессе генерации текста или для вычисления потерь во время обучения.

In [388]:
import torch
import torch.nn as nn
from transformers import GPT2Model

class CustomGPT2Model(nn.Module):
    def __init__(
        self, 
        pretrained_model_name,
        num_message_types,
        data_path=Path('')
    ):
        super(CustomGPT2Model, self).__init__()
        # Загрузка предобученной модели GPT-2
        self.gpt2 = GPT2Model.from_pretrained(pretrained_model_name, cache_dir=data_path)
        
        # Слой для встраивания типа сообщения
        self.type_embedding = nn.Embedding(num_message_types, self.gpt2.config.hidden_size)
        
        # Дополнительный линейный слой для объединения встраивания типа сообщения и токенов
        self.combined_linear = nn.Linear(self.gpt2.config.hidden_size, self.gpt2.config.hidden_size)
        # Дополнительный слой для логитов
        self.lm_head = nn.Linear(self.gpt2.config.hidden_size, self.gpt2.config.vocab_size)  # Дополнительный слой для логитов


    def forward(self, input_ids, attention_mask, type_ids):
        # Получение встраиваний токенов
        inputs_embeds = self.gpt2.wte(input_ids)  # wte - word token embeddings
        
        # Получение встраивания для типа сообщения
        type_embeds = self.type_embedding(type_ids).unsqueeze(1)  # Расширяем размерности для сложения
        
        # Сложение встраиваний токенов и типа по всей длине входа
        combined_embeds = inputs_embeds + type_embeds.expand(-1, input_ids.size(1), -1)
        
        # Применяем дополнительный линейный слой
        combined_embeds = self.combined_linear(combined_embeds)
        
        # Передача встраиваний в основную модель GPT-2
        outputs = self.gpt2(inputs_embeds=combined_embeds, attention_mask=attention_mask)
        
        logits = self.lm_head(outputs.last_hidden_state)
        return logits


# Оценка

In [389]:
def save_plt_img(save_path, fig_id, tight_layout=True, fig_extension="png", resolution=300):
        path = save_path / f"{fig_id}.{fig_extension}"
        if tight_layout:
            plt.tight_layout()
        plt.savefig(path, format=fig_extension, dpi=resolution) 

In [390]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
from torch.utils.data import DataLoader


class Evaluator:
    def __init__(self, tokenizer, weights={'bleu': 0.34, 'rouge': 0.33, 'meteor': 0.33}):
        self.tokenizer = tokenizer
        self.weights = weights
        self.rouge = Rouge()
        
        self.metrics = []

    def evaluate(self, references: list, hypotheses: list):
        tokenized_references = [[ref.split()] for ref in references]
        tokenized_hypotheses = [hyp.split() for hyp in hypotheses]
        
        # Вычисление метрик BLEU, ROUGE, METEOR
        bleu_score = corpus_bleu(tokenized_references, tokenized_hypotheses)
        
        rouge_score = self.rouge.get_scores(hypotheses, references, avg=True)['rouge-l']['f']
        
        list_meteor_score = [meteor_score(refs, hyp) for refs, hyp in zip(tokenized_references, tokenized_hypotheses)]
        avg_meteor_score = np.mean(list_meteor_score)

        # Словарь с результатами
        results = {
            'overall': self.weights['bleu'] * bleu_score +
                       self.weights['rouge'] * rouge_score +
                       self.weights['meteor'] * avg_meteor_score,
            'bleu': bleu_score,
            'rouge': rouge_score,
            'meteor': avg_meteor_score
        }
        self.metrics.append(results)
        return results
    
    def plot_metrics(self, imgs_path=Path('')):
        epochs_range = range(1, len(self.metrics) + 1)
        fig = plt.figure(figsize=(15, 6))  # Устанавливаем размер фигуры
        
        ax12 = plt.subplot(1, 2, 1)
        ax12.plot(epochs_range, [m['overall'] for m in self.metrics], label='Overall Score', color='tab:green')
        ax12.set_title('Overall Evaluation Score')
        ax12.set_xlabel('Epochs')
        ax12.set_ylabel('Score')
        ax12.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax12.legend(loc='upper right')

        # Отдельные графики для каждой метрики
        ax21 = plt.subplot(1, 2, 2)
        ax21.plot(epochs_range, [m['bleu'] for m in self.metrics], label='BLEU Score', color='tab:red')
        ax21.plot(epochs_range, [m['rouge'] for m in self.metrics], label='ROUGE Score', color='tab:pink')
        ax21.plot(epochs_range, [m['meteor'] for m in self.metrics], label='METEOR Score', color='tab:brown')
        ax21.set_title('Individual Metrics')
        ax21.set_xlabel('Epochs')
        ax21.set_ylabel('Score')
        ax21.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax21.legend(loc='upper right')      

        fig.tight_layout()  # Убедимся, что макет не нарушен
        save_plt_img(imgs_path, "spacial_metrics")
        plt.show()


# Конфиг

In [391]:
import torch

class Config:
    model_name = 'ai-forever/rugpt3small_based_on_gpt2'
    max_length = 32
    temperature=0.9
    batch_size = 16
    test_size=0.1
    learning_rate = 1e-5
    num_epochs = 3
    uniq_name = 'custome_gpt2_model'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Тренер

In [392]:
from torch import Tensor
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn.functional as F
from tqdm.auto import tqdm


class Trainer:
    def __init__(
        self, 
        model: CustomGPT2Model, 
        evaluator: Evaluator, 
        tokenizer,
        learning_rate=1e-3,
        special_eval=False,
        device='cpu',
        imgs_path=Path(''),
        config: Config=None
    ):
        self.model = model.to(device)
        self.special_eval = special_eval
        self.device = device
        self.evaluator =  evaluator
        self.tokenizer = tokenizer
        self.imgs_path = imgs_path
        self.config = config
        
        self.train_losses = []
        self.val_losses = []
        
        self.optimizer = AdamW(model.parameters(), lr=learning_rate)
        self.criterion = CrossEntropyLoss()
        
        # self.freeze_layers()
        # self.print_freezed_layers()

    def train(self, train_loader: DataLoader):
        """
        Обучение модели
        """
        self.model.train()
        train_loss = 0.0
        train_bar = tqdm(train_loader, desc='Training', leave=True)
        for batch in train_bar:
            inputs_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            target_ids = batch['target_ids'].to(self.device)
            type_ids = batch['type_ids'].to(self.device)
            
            logits = self.model(input_ids=inputs_ids, attention_mask=attention_mask, type_ids=type_ids)
            loss = self.criterion(
                logits.view(-1, self.model.gpt2.config.vocab_size),
                target_ids.view(-1)
            )
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            train_loss += loss.item()
            train_bar.set_postfix({'train loss': loss.item()})
            
        avg_loss = train_loss / len(train_loader)
        self.train_losses.append(avg_loss)        
        return avg_loss
            

    def eval(self, eval_loader: DataLoader):
        self.model.eval()
        eval_loss = 0
        eval_bar = tqdm(eval_loader, desc=f'Evaluate', leave=True)
        with torch.no_grad():
            for batch in eval_bar:
                inputs_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                target_ids = batch['target_ids'].to(self.device)
                type_ids = batch['type_ids'].to(self.device)
                
                logits = self.model(input_ids=inputs_ids, attention_mask=attention_mask, type_ids=type_ids)

                loss = self.criterion(
                    logits.view(-1, self.model.gpt2.config.vocab_size),
                    target_ids.view(-1)
                )                          
                eval_loss += loss.item()
                eval_bar.set_postfix({'eval loss': loss.item()})
                
            avg_loss = eval_loss / len(eval_loader)
            self.val_losses.append(avg_loss)    
        
        self.calculate_special_eval(eval_loader, self.config.max_length, self.config.temperature)
        
        return avg_loss
            
    def fit(self, epochs, train_loader: DataLoader, eval_loader: DataLoader):
        epoch_bar = tqdm(range(epochs), desc=f'Fit ', leave=True)
        for epoch in epoch_bar:
             # Вызов функции обучения и получение средней потери на тренировочных данных
            train_avg_loss = self.train(train_loader)
            # Вызов функции оценки и получение средней потери на валидационных данных
            eval_avg_loss = self.eval(eval_loader)
            
            # Обновление прогресс-бара с текущей эпохой и значениями потерь
            epoch_bar.set_postfix({
                'Epoch': epoch,
                'Train Loss': f"{train_avg_loss:.4f}",
                'Eval Loss': f"{eval_avg_loss:.4f}"
            })
        
    def calculate_special_eval(self, eval_loader: DataLoader, max_length, temperature):
        if self.special_eval and self.evaluator:
            self.model.eval()
            eval_bar = tqdm(eval_loader, desc=f'Special evaluate', leave=True)
            hypotheses = []
            references = []
            
            with torch.no_grad():
                for batch in eval_bar:
                    input_ids = batch['input_ids'].to(self.device)
                    attention_mask = batch['attention_mask'].to(self.device)
                    target_ids = batch['target_ids'].to(self.device)
                    type_ids = batch['type_ids'].to(self.device)
                    
                    hypotheses += self._generate_text_sampling(input_ids, attention_mask, type_ids, max_length, temperature)
                    references += self.decode_tensor(target_ids)
            self.evaluator.evaluate(hypotheses=hypotheses, references=references)
        
    def generate_text_sampling(self, text, type_id, max_length=25, temperature=0.9):        
        # Токенизация входного текста
        inputs = self.tokenizer(text, return_tensors="pt")
        input_ids = inputs['input_ids'].to(self.device)
        attention_mask = inputs['attention_mask'].to(self.device)
        type_ids = torch.tensor([type_id]).to(self.device)
        
        return self._generate_text_sampling(input_ids, attention_mask, type_ids, max_length, temperature)
    
    def _generate_text_sampling(self, input_ids, attention_mask, type_ids, max_length=25, temperature=0.9):
        self.model.eval() 
        generated_sequence = input_ids   
        
        # Начальная длина входных данных
        start_gen_index = input_ids.size(1) 
        
        with torch.no_grad():
            for _ in range(max_length):
                outputs = self.model(input_ids=generated_sequence, attention_mask=attention_mask, type_ids=type_ids)
                logits = outputs[:, -1, :]                
                # Применяем температуру для управления случайностью выборки
                logits = logits / temperature
                probabilities = F.softmax(logits, dim=-1)
                next_token_id = torch.multinomial(probabilities, num_samples=1)                
                generated_sequence = torch.cat((generated_sequence, next_token_id), dim=-1)
                
                # Обновление attention_mask
                attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.size(0), 1), device=self.device)], dim=1)
                
                # if next_token_id.item() == self.tokenizer.eos_token_id:
                #     break
        # Возвращаем только сгенерированную часть
        generated_part = generated_sequence[:, start_gen_index:]
        return self.decode_tensor(generated_part)
    
    def decode_tensor(self, sequences: Tensor):
        decoded_texts = []
        for sequence in sequences:
            decoded_text = self.tokenizer.decode(sequence, skip_special_tokens=True)
            decoded_texts.append(decoded_text)
        return decoded_texts

        # return self.tokenizer.decode(sequence.squeeze(), skip_special_tokens=True)

    def plot_main_metrics(self, imgs_path):
        epochs_range = range(1, len(self.val_losses) + 1)
        fig = plt.figure(figsize=(15, 6))  # Устанавливаем размер фигуры
        
        ax11 = plt.subplot()
        ax11.plot(epochs_range, self.train_losses, label='Train loss', color='tab:red')
        ax11.plot(epochs_range, self.val_losses, label='Evaluate loss', color='tab:blue')
        ax11.set_title('Losses over Epochs')
        ax11.set_xlabel('Epochs')
        ax11.set_ylabel('Loss')
        # ax11.tick_params(axis='y', labelcolor='tab:red')
        ax11.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax11.legend(loc='upper right')     

        fig.tight_layout()  # Убедимся, что макет не нарушен
        save_plt_img(imgs_path, "train_metrics")  # extra code
        plt.show()
            
    def freeze_layers(self, num_trainable_blocks=2):        
        total_blocks = len(self.model.gpt2.h)  # h - это список всех блоков трансформера в модели GPT-2

        # Заморозка слоёв в начальных блоках
        for i, block in enumerate(self.model.gpt2.h):
            if i < total_blocks - num_trainable_blocks:
                for param in block.parameters():
                    param.requires_grad = False
            else:
                for param in block.parameters():
                    param.requires_grad = True

        # # Если у вас есть другие специфические слои, которые нужно обучать, размораживаем их
        # for param in model.combined_linear.parameters():
        #     param.requires_grad = True
    
    def print_freezed_layers(self):
        for name, param in self.model.named_parameters():
            print(f"{name} is trainable: {param.requires_grad}")

In [393]:

from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer


class TrainingManager:
    def __init__(self, 
        dataframe: pd.DataFrame,
        categorical_encoder: CategoricalLabelEncoder,
        config: Config,
        data_path: Path=DATA_PATH,
        imgs_path: Path=IMAGES_PATH 
    ):
        
        self.uniq_name = config.uniq_name
        self.data_path = data_path 
        self.imgs_path = imgs_path    
        self.dataframe = dataframe
        self.categorical_encoder = categorical_encoder
                
        self.config = config
        
        self.tokenizer = GPT2Tokenizer.from_pretrained(config.model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Разбиение данных на обучающую и тестовую выборки
        train_data, eval_data = train_test_split(
            self.dataframe[['type_problem', 'source', 'target']], test_size=0.1
        )

        # Создание объектов Dataset
        train_dataset = CustomDataset(dataframe=train_data, tokenizer=self.tokenizer, max_length=config.max_length)
        eval_dataset = CustomDataset(dataframe=eval_data, tokenizer=self.tokenizer, max_length=config.max_length)

        self.train_dataloader = DataLoader(
            train_dataset, 
            batch_size=config.batch_size, 
            shuffle=True
        )
        self.eval_dataloader = DataLoader(
            eval_dataset, 
            batch_size=config.batch_size, 
            shuffle=False
        )

        self.model = CustomGPT2Model(
            pretrained_model_name=config.model_name,
            num_message_types=len(self.categorical_encoder.get_classes('type_problem')),
            data_path=self.data_path
        )
        
        evaluator = Evaluator(
            self.tokenizer 
        )
        
        self.trainer = Trainer(
            model=self.model,
            evaluator=evaluator,
            tokenizer=self.tokenizer,
            learning_rate=config.learning_rate,
            device=config.device,
            imgs_path=self.imgs_path,
            config=config   
        )  
    
    def fit(self):
        """
        Запуск процесса обучения.
        """
        
        self.trainer.fit(self.config.num_epochs, self.train_dataloader, self.eval_dataloader)
        self.trainer.plot_main_metrics(self.imgs_path)
        # self.trainer.evaluator.plot_metrics(self.imgs_path)
        
    def generate_text_sampling(self, text, type_id, max_length=25, temperature=0.9):
        return self.trainer.generate_text_sampling(text, type_id, max_length, temperature)[0]


    # def save(self):
    #     self._save_model()
    #     self._save_standard_scalar()
 
    # def load(self):
    #     self._load_model()
    #     self._load_standard_scalar()
    
    # def _save_model(self):
    #     """
    #     Сохранение обученной модели.        
    #     """
    #     self.trainer.save_model(self.data_path+self.uniq_name+'_model_weight.pth')
        
    # def _load_model(self):
    #     """
    #     Загрузка обученной модели.        
    #     """
    #     self.trainer = Trainer.load_model(self.data_path+self.uniq_name+'_model_weight.pth')


# Гиперпараметры

In [394]:
work_dataframe = work_dataframe[:5000]

In [395]:
config = Config()

config.learning_rate = 1e-3
config.num_epochs = 5
config.max_length = 128
config.temperature = 0.9
config.batch_size = 16

# Обучение

In [396]:
training_manager = TrainingManager(
    work_dataframe,
    datafarme_encoders,
    config
)

Список категорий в 'categoria': ['Благодарности' 'Комментарии' 'Критика власти' 'Неопределенное сообщение'
 'Ответы' 'Повторная жалоба' 'Предложение/инициатива/идея'
 'Просьба проинформировать' 'Семьи военнослужащих' 'Соболезнования'
 'Устранение проблемы']


d:\Projects\NLP_course\.venv\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [397]:
training_manager.fit()

Fit :   0%|          | 0/5 [00:00<?, ?it/s]

Training:   0%|          | 0/282 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Предикт

In [ ]:
testing_data: pd.DataFrame = training_manager.eval_dataloader.dataset.dataframe
testing_data = testing_data.reset_index()
testing_data.head()

,index,type_problem,source,target
0,1558,10,Андрей Юрьевич! Доброго времени суток.Село Кос...,"Добрый день, Ваше обращение направлено специал..."
1,2678,10,Всем добрый вечер.\n\nПосле снятия гипса со сл...,Добрый день. Ваше обращение передано в работу.
2,2745,10,Розыгрышей я не видела \nГорячих напитков не б...,"[id59389450|Татьяна], Добрый день! Для гостей,..."
3,1097,10,У нас в Ликино-Дулëво проехать попробовали. Эт...,[id569706568|Сергей Иконников]\nЗдравствуйте! ...
4,723,10,"Андрей Юрьевич, просим разобраться в ситуации!...",[id39192224|Ольга Руденко] Добрый день. Измене...


In [ ]:

type_message = testing_data['type_problem'][0]
source = testing_data['source'][0]
target = testing_data['target'][0]

print(datafarme_encoders.decode('type_problem', type_message))

print('Source - ', source)
print('Target - ', target)


['Устранение проблемы']
Source -  Андрей Юрьевич! Доброго времени суток.Село Костино Дмитровский городской округ,    частный сектор обслуживание территориальным отделом номер 6  Сергеем Колковым. У нас проживают семьи с детьми. Дорогу не чистят уже 2 день, ещё  дорога очень узкая мы очень просим прислать нам грейдер потому что трактор не справляется уже с такой нагрузкой,снега очень много. А дорогу постоянно переметает с поля. Как детям пробираться в школу по заметённой дороге? Пожалуйста примите меры. Спасибо.56.318002, 37.716737 координаты территории
Target -  Добрый день, Ваше обращение направлено специалистам, мы вернёмся позже с ответом, как только получим информацию.
С Уважением, МЦУР Дмитровского г. о.


In [ ]:
print(training_manager.generate_text_sampling(source, type_message, max_length=16, temperature=1))

оль. грим49527 поможет направленоНам д позже г. МО.os


# Выполнение

# Решение